<a href="https://colab.research.google.com/github/PanyPy/LAPD-Crime-Data-Analysis/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install census
!pip install osmnx
!pip install us


In [ ]:
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import os
import branca.colormap as cm
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from shapely.geometry import Point
from scipy.spatial import cKDTree
from census import Census
from imblearn.pipeline import Pipeline as ImbPipeline
from us import states
from sklearn.cluster import DBSCAN, KMeans
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc, silhouette_score, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

pd.set_option('display.max_columns', None)

**Data Demografica**

Solicitud de API Key: https://api.census.gov/data/key_signup.html

In [ ]:
c = Census("918dc228d51aa92a07f0540682661baaf0be44e2")

# Lista de años de interés
years = list(range(2010, 2023))

# FIPS code for Los Angeles County
county_fips = '037'
state_fips = states.CA.fips

# Variables de interés, incluyendo la población hispana o latina
variables = ('NAME', 'B01003_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 'B03001_003E', 'B19013_001E')

# Lista para almacenar los DataFrames anuales
dfs = []

# Obtener datos para cada año
for year in years:
    data = c.acs5.state_county_subdivision(variables, state_fips, county_fips, '*', year=year)
    df = pd.DataFrame(data)
    df['Year'] = year
    dfs.append(df)

# Concatenar todos los DataFrames anuales en uno solo
demographic_data = pd.concat(dfs, ignore_index=True)

# Renombrar las columnas para mayor claridad
demographic_data = demographic_data.rename(columns={
    'NAME': 'Area',
    'B01003_001E': 'Total Population',
    'B02001_002E': 'White',
    'B02001_003E': 'Black or African American',
    'B02001_004E': 'American Indian and Alaska Native',
    'B02001_005E': 'Asian',
    'B03001_003E': 'Hispanic or Latino',
    'B19013_001E': 'Median Household Income'
})

# Mostrar el DataFrame consolidado
demographic_data

,Area,Total Population,White,Black or African American,American Indian and Alaska Native,Asian,Hispanic or Latino,Median Household Income,state,county,county subdivision,Year
0,"Agoura Hills-Malibu CCD, Los Angeles County, C...",63257.0,53261.0,1215.0,217.0,4121.0,5243.0,112837.0,06,037,90015,2010
1,"Compton CCD, Los Angeles County, California",339005.0,112205.0,86630.0,1706.0,31027.0,196150.0,50837.0,06,037,90560,2010
2,"Downey-Norwalk CCD, Los Angeles County, Califo...",412456.0,171327.0,29209.0,2257.0,71668.0,239863.0,57521.0,06,037,90730,2010
3,"East San Gabriel Valley CCD, Los Angeles Count...",937108.0,444781.0,32791.0,4913.0,181389.0,502850.0,64566.0,06,037,90810,2010
4,"Inglewood CCD, Los Angeles County, California",366684.0,112080.0,112027.0,1134.0,29740.0,186764.0,44052.0,06,037,91400,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
255,"South Gate-East Los Angeles CCD, Los Angeles C...",477554.0,163488.0,7496.0,8251.0,4731.0,454239.0,59962.0,06,037,93155,2022
256,"Southwest San Gabriel Valley CCD, Los Angeles ...",314362.0,55013.0,4059.0,3462.0,157326.0,124657.0,77771.0,06,037,93200,2022
257,"Torrance CCD, Los Angeles County, California",145454.0,56586.0,4879.0,841.0,55048.0,28062.0,109554.0,06,037,93380,2022
258,"Upper San Gabriel Valley CCD, Los Angeles Coun...",321072.0,92912.0,6351.0,4619.0,112568.0,144806.0,83928.0,06,037,93510,2022


La relacion de esta tabla con el resto vendra por la columna "county subdivision" y "year"

**Data de Crimenes**

In [ ]:
# Leer los dos conjuntos de datos
crime_2010_2019 = pd.read_csv(r'Crime_Data_from_2010_to_2019.csv')
crime_2020_present = pd.read_csv(r'Crime_Data_from_2020_to_Present.csv')

# Renombrar la columna 'AREA ' en el primer DataFrame
crime_2010_2019.rename(columns={'AREA ': 'AREA'}, inplace=True)

# Concatenar los DataFrames
crime_data = pd.concat([crime_2010_2019, crime_2020_present], ignore_index=True)

# Calcular el porcentaje de valores nulos por columna
total_rows = len(crime_data)  # Obtener el número total de filas en el DataFrame
null_info = crime_data.isnull().sum()
null_percentage = (null_info  / total_rows) * 100  # Calcular el porcentaje de valores nulos por columna

# Concatenar las series null_counts y null_percentage en un DataFrame
null_info = pd.concat([null_info , null_percentage], axis=1, keys=['Número de Nulls', 'Porcentaje de Nulls'])

# Mostrar los resultados
print("Información sobre valores nulls por columna:")
#print(null_info.sort_values(by='Porcentaje de Nulos', ascending=False))

Información sobre valores nulos por columna:


**Data de arrestos**

In [ ]:
arrest_2010_2019 = pd.read_csv(r'Arrest_Data_from_2010_to_2019.csv')
arrest_2020_present = pd.read_csv(r'Arrest_Data_from_2020_to_Present.csv')
arrest_data = pd.concat([arrest_2010_2019, arrest_2020_present], ignore_index=True)

# Calcular el porcentaje de valores nulos por columna
total_rows = len(arrest_data)  # Obtener el número total de filas en el DataFrame
null_info = arrest_data.isnull().sum()
null_percentage = (null_info  / total_rows) * 100  # Calcular el porcentaje de valores nulos por columna

# Concatenar las series null_counts y null_percentage en un DataFrame
null_info = pd.concat([null_info , null_percentage], axis=1, keys=['Número de Nulos', 'Porcentaje de Nulos'])

# Mostrar los resultados
print("Información sobre valores nulos por columna:")
#print(null_info.sort_values(by='Porcentaje de Nulos', ascending=False))

Información sobre valores nulos por columna:


In [ ]:
# Ensure the date columns are in datetime format
crime_data['date'] = pd.to_datetime(crime_data['Date Rptd'])
arrest_data['date'] = pd.to_datetime(arrest_data['Arrest Date'])

# Extraer el año y el mes a nuevas columnas
crime_data['year'] = crime_data['date'].dt.year
crime_data['month'] = crime_data['date'].dt.month
crime_data['day'] = crime_data['date'].dt.day

arrest_data['year'] = arrest_data['date'].dt.year
arrest_data['month'] = arrest_data['date'].dt.month
arrest_data['day'] = arrest_data['date'].dt.day

Data de county subdivisions viene del mismo CENSUS, fue extraida en este link: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2023&layergroup=County+Subdivisions
y convertida a geojson.

In [ ]:
# Create GeoDataFrames
crime_gdf = gpd.GeoDataFrame(crime_data, geometry=gpd.points_from_xy(crime_data.LON, crime_data.LAT))
arrest_gdf = gpd.GeoDataFrame(arrest_data, geometry=gpd.points_from_xy(arrest_data.LON, arrest_data.LAT))

# Extraemos informacion sobre los subdivisions
county_subdivisions_gdf = gpd.read_file('tl_2023_06_cousub.geojson')

# Asegurarse de que ambos GeoDataFrames están en el mismo sistema de referencia de coordenadas (CRS)
crimes_gdf = crime_gdf.set_crs(county_subdivisions_gdf.crs, allow_override=True)
arrests_gdf = arrest_gdf.set_crs(county_subdivisions_gdf.crs, allow_override=True)

# Realizar la unión espacial para agregar la subdivisión del condado a los datos de crímenes
crimes_with_subdivision = gpd.sjoin(crimes_gdf, county_subdivisions_gdf, how='left', op='within')

# Realizar la unión espacial para agregar la subdivisión del condado a los datos de arrestos
arrests_with_subdivision = gpd.sjoin(arrests_gdf, county_subdivisions_gdf, how='left', op='within')

# Opcional: Convertir de nuevo a DataFrame de pandas si no necesitas la geometría
crime_data = pd.DataFrame(crimes_with_subdivision.drop(columns='geometry'))
arrest_data = pd.DataFrame(arrests_with_subdivision.drop(columns='geometry'))

C:\Users\andre\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\andre\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


**Data Geoespacial**

In [ ]:
# Fetch schools and alcohol-selling stores data using osmnx
place_name = "Los Angeles, California, USA"

# Schools
schools_gdf = ox.geometries_from_place(place_name, tags={'amenity': 'school'})
schools_gdf = schools_gdf[schools_gdf.geometry.type == 'Point']

# Alcohol-selling stores
alcohol_stores_gdf = ox.geometries_from_place(place_name, tags={'shop': 'alcohol'})
alcohol_stores_gdf = alcohol_stores_gdf[alcohol_stores_gdf.geometry.type == 'Point']

# Parques
parks_gdf = ox.geometries_from_place(place_name, tags={'leisure': 'park'})
parks_gdf = parks_gdf[parks_gdf.geometry.type == 'Point']

# Estaciones de Policía
police_stations_gdf = ox.geometries_from_place(place_name, tags={'amenity': 'police'})
police_stations_gdf = police_stations_gdf[police_stations_gdf.geometry.type == 'Point']

# Hospitales
hospitals_gdf = ox.geometries_from_place(place_name, tags={'amenity': 'hospital'})
hospitals_gdf = hospitals_gdf[hospitals_gdf.geometry.type == 'Point']

# Estaciones de Transporte Público
transit_stations_gdf = ox.geometries_from_place(place_name, tags={'public_transport': 'station'})
transit_stations_gdf = transit_stations_gdf[transit_stations_gdf.geometry.type == 'Point']

# Zonas Comerciales
retail_areas_gdf = ox.geometries_from_place(place_name, tags={'landuse': 'retail'})
retail_areas_gdf = retail_areas_gdf[retail_areas_gdf.geometry.type == 'Point']

# Bares
bars_gdf = ox.geometries_from_place(place_name, tags={'amenity': 'bar'})
bars_gdf = bars_gdf[bars_gdf.geometry.type == 'Point']

# Restaurantes
restaurants_gdf = ox.geometries_from_place(place_name, tags={'amenity': 'restaurant'})
restaurants_gdf = restaurants_gdf[restaurants_gdf.geometry.type == 'Point']

# Áreas Residenciales
residential_areas_gdf = ox.geometries_from_place(place_name, tags={'landuse': 'residential'})
residential_areas_gdf = residential_areas_gdf[residential_areas_gdf.geometry.type == 'Point']

# Convertimos a dataframes
schools = pd.DataFrame({'LAT': schools_gdf.geometry.y, 'LON': schools_gdf.geometry.x})
alcohol_stores = pd.DataFrame({'LAT': alcohol_stores_gdf.geometry.y, 'LON': alcohol_stores_gdf.geometry.x})
parks = pd.DataFrame({'LAT': parks_gdf.geometry.y, 'LON': parks_gdf.geometry.x})
police_stations = pd.DataFrame({'LAT': police_stations_gdf.geometry.y, 'LON': police_stations_gdf.geometry.x})
hospitals = pd.DataFrame({'LAT': hospitals_gdf.geometry.y, 'LON': hospitals_gdf.geometry.x})
transit_stations = pd.DataFrame({'LAT': transit_stations_gdf.geometry.y, 'LON': transit_stations_gdf.geometry.x})
retail_areas = pd.DataFrame({'LAT': retail_areas_gdf.geometry.y, 'LON': retail_areas_gdf.geometry.x})
residential_areas = pd.DataFrame({'LAT': residential_areas_gdf.geometry.y, 'LON': residential_areas_gdf.geometry.x})
bars = pd.DataFrame({'LAT': bars_gdf.geometry.y, 'LON': bars_gdf.geometry.x})
restaurants = pd.DataFrame({'LAT': restaurants_gdf.geometry.y, 'LON': restaurants_gdf.geometry.x})


C:\Users\andre\AppData\Local\Temp\ipykernel_18724\2957169652.py:5: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  schools_gdf = ox.geometries_from_place(place_name, tags={'amenity': 'school'})
C:\Users\andre\AppData\Local\Temp\ipykernel_18724\2957169652.py:9: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  alcohol_stores_gdf = ox.geometries_from_place(place_name, tags={'shop': 'alcohol'})
C:\Users\

**Crime Map**

In [ ]:
# # Filtrar datos de crímenes para excluir filas con NaN en Crm Cd 1
# crime_data = crime_data.dropna(subset=['Crm Cd 1'])

# # Crear una carpeta para guardar los mapas
# output_folder = 'crime_maps_by_year'
# os.makedirs(output_folder, exist_ok=True)

# # Generar un mapa para cada año
# for year, year_data in crime_data.groupby('year'):
#     # Inicializar el mapa centrado en las coordenadas promedio de los crímenes del año
#     m = folium.Map(location=[year_data.LAT.mean(), year_data.LON.mean()], zoom_start=12)

#     # Añadir el heatmap de crímenes con intensidad basada en Crm Cd 1
#     crime_heat_data = [
#         [row['LAT'], row['LON'], max(1 / (row['Crm Cd 1'] + 1), 0.1)] for index, row in year_data.iterrows()
#     ]
#     HeatMap(crime_heat_data, radius=10, name="Crime Heatmap", gradient={0.4: 'lime', 0.65: 'purple', 1: 'red'}).add_to(m)

#     # Añadir las subdivisiones del condado al mapa con nombres
#     geojson = folium.GeoJson(
#         county_subdivisions_gdf,
#         name="County Subdivisions",
#         style_function=lambda x: {
#             'fillColor': 'transparent',
#             'color': 'blue',
#             'weight': 2
#         },
#         tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['Subdivision:'])
#     )
#     geojson.add_to(m)

#     # Añadir las estaciones de policía al mapa
#     for idx, row in police_stations.iterrows():
#         folium.Marker([row['LAT'], row['LON']], popup='Police Station', icon=folium.Icon(color='lightblue')).add_to(m)

#     # Añadir las tiendas de alcohol al mapa
#     for idx, row in alcohol_stores.iterrows():
#         folium.Marker([row['LAT'], row['LON']], popup='Alcohol Store', icon=folium.Icon(color='red')).add_to(m)

#     # Añadir control de capas
#     folium.LayerControl().add_to(m)

#     # Guardar el mapa en un archivo HTML con el año en el nombre del archivo
#     map_filename = os.path.join(output_folder, f'crime_hotzones_map_{year}.html')
#     m.save(map_filename)
#     print(f'Map for {year} saved to {map_filename}')

Vamos a separar los niveles de gravedad del crimen en 3 categorias:
- Categoria Grave **(nivel 1)**:
      RAPE,
      ATTEMPTED
      RAPE
      FORCIBLE
      CRIMINAL HOMICIDE
      MANSLAUGHTER
      NEGLIGENT
- Categoria Semi - grave **(nivel 2)**:
      ASSAULT WITH DEADLY WEAPON
      AGGRAVATED ASSAULT
      ROBBERY
      CHILD NEGLECT (SEE 300 W.I.C.)
      INTIMATE PARTNER - AGGRAVATED ASSAULT
      ATTEMPTED ROBBERY
      SHOTS FIRED AT INHABITED DWELLING
      CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT
      ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER
      SHOTS FIRED AT MOVING VEHICLE
      TRAIN OR AIRCRAFT
- Categoria No tan grave **(nivel 3)**: resto de crimenes.

In [ ]:
crime_data['crime_gravity'] = np.where(crime_data['Crm Cd'] < 200, 1,
                                       np.where(crime_data['Crm Cd'] < 300, 2, 3))

**Modelo DBSCAN**

In [ ]:
# Nos quedamos con la informacion desde el 2023
crime_data = crime_data.loc[crime_data['year'] <= 2023]

# Asegurarse de que la columna 'county subdivision' sea de tipo string
demographic_data['county subdivision'] = demographic_data['county subdivision'].astype(str)
crime_data['county subdivision'] = crime_data['COUSUBFP'].astype(str)

# Unir los datos de crímenes con los datos demográficos basados en la subdivisión del condado y el año
crime_data = crime_data.merge(demographic_data, left_on=['county subdivision', 'year'], right_on=['county subdivision', 'Year'], how='left')

In [ ]:
# Función para contar locales de alcohol cercanos a cada crimen, radio de 1.1 Km
def count_nearby_alcohol(crimenes_df, alcohol_df, radius=0.01):
    crime_coords = crimenes_df[['LAT', 'LON']].values
    alcohol_coords = alcohol_df[['LAT', 'LON']].values
    tree = cKDTree(alcohol_coords)
    counts = tree.query_ball_point(crime_coords, radius)
    alcohol_counts = [len(c) for c in counts]
    return alcohol_counts

# Agregar conteo de locales de alcohol cercanos
crime_data['number_of_alcohol_stores_nearby'] = count_nearby_alcohol(crime_data, alcohol_stores)

# Obtenemos una variable de 1 si hay stores cercanos o de 0 de no ser asi
crime_data['alcohol_stores_nearby'] = np.where(crime_data['number_of_alcohol_stores_nearby'] > 0, 1, 0)

In [ ]:
# crime_data = pd.read_parquet('crime_data.parquet')
# demographic_data = pd.read_parquet('demographic_data.parquet')
# alcohol_stores = pd.read_parquet('alcohol_stores.parquet')

In [ ]:
# Trabajar con una muestra más pequeña
alcohol_stores = alcohol_stores[['LAT', 'LON']].reset_index()

# Preparar los datos
features = alcohol_stores[['LAT', 'LON']]

# Normalizar las características
scaler = StandardScaler()
X = scaler.fit_transform(features)

# Aplicar DBSCAN
dbscan = DBSCAN(eps=0.7, min_samples=12)
labels = dbscan.fit_predict(X)

# Añadir los clústeres al dataframe original
alcohol_stores['cluster_dbscan'] = labels

# Evaluar la cantidad de clústeres y noise points
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)
print(f'Estimated number of clusters: {n_clusters}')
print(f'Estimated number of noise points: {n_noise}')

Estimated number of clusters: 3
Estimated number of noise points: 3


In [ ]:
# Probar diferentes combinaciones de eps y min_samples manualmente
eps_values = [0.6, 0.7, 0.8, 0.9]
min_samples_values = [5, 12, 10, 15]

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X)

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)

        if n_clusters > 2:
            print(f"eps: {eps}, min_samples: {min_samples}, clusters: {n_clusters}, noise points: {n_noise}")
            break
    if n_clusters > 1:
        break

# Aplicar DBSCAN con los mejores parámetros encontrados
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X)

# Añadir los clústeres al dataframe original
alcohol_stores['cluster_dbscan'] = labels

# Evaluar la cantidad de clústeres y puntos de ruido
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)
print(f'Estimated number of clusters: {n_clusters}')
print(f'Estimated number of noise points: {n_noise}')

eps: 0.7, min_samples: 12, clusters: 3, noise points: 3
Estimated number of clusters: 3
Estimated number of noise points: 3


In [ ]:
# Visualizar los clústeres en un mapa folium
map_folium = folium.Map(location=[alcohol_stores['LAT'].mean(), alcohol_stores['LON'].mean()], zoom_start=12)

# Crear una superposición de colores para los clústeres
unique_clusters = n_clusters
colors = sns.color_palette("hsv", unique_clusters)
colormap = cm.LinearColormap(colors=colors, vmin=0, vmax=unique_clusters-1)

for idx, row in alcohol_stores.iterrows():
    if row['cluster_dbscan'] != -1:  # Excluir puntos de ruido
        color = colormap(row['cluster_dbscan'])
    else:
        color = 'black'  # Color para los puntos de ruido
    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Cluster: {row['cluster_dbscan']}"
    ).add_to(map_folium)

# Añadir la leyenda
colormap.add_to(map_folium)

# Guardar el mapa
#map_folium.save('dbscan_manual_clusters_map.html')
map_folium

**Modelo K-means**

In [ ]:
# Normalizar las características
scaler = StandardScaler()
X = scaler.fit_transform(features)

# Aplicar KMeans para obtener un número específico de clústeres
n_clusters = 5  # Ajusta este valor según sea necesario
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X)

# Añadir los clústeres al dataframe original
alcohol_stores['cluster_k_means'] = labels

# Evaluar la cantidad de clústeres
unique_clusters = len(set(labels))
print(f'Estimated number of clusters: {unique_clusters}')

# Visualizar los clústeres en un mapa folium
map_folium = folium.Map(location=[alcohol_stores['LAT'].mean(), alcohol_stores['LON'].mean()], zoom_start=12)

# Crear una superposición de colores para los clústeres
colors = sns.color_palette("hsv", unique_clusters)
colormap = cm.LinearColormap(colors=colors, vmin=0, vmax=unique_clusters-1)

for idx, row in alcohol_stores.iterrows():
    color = colormap(row['cluster_k_means'])
    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Cluster: {row['cluster_k_means']}"
    ).add_to(map_folium)

# Añadir la leyenda
colormap.add_to(map_folium)

# Guardar el mapa
#map_folium.save('kmeans_clusters_map.html')
map_folium


Estimated number of clusters: 5


Parece que k-means nos ayuda mejor a separar en clusters los alcohol stores, por lo que ser aque que consideraremos de ahora en adelante.

**Distancias (en metros) con Alcohol Stores**

In [ ]:
# Crear GeoSeries para las escuelas, tiendas de alcohol y estaciones de policía
alcohol_stores['LON'] = alcohol_stores['LON'].astype(float)
alcohol_stores['LAT'] = alcohol_stores['LAT'].astype(float)
alcohol_stores_gs = gpd.GeoSeries([Point(xy) for xy in zip(alcohol_stores['LON'], alcohol_stores['LAT'])], crs="EPSG:4326")

# Crear una GeoSeries para los crímenes y arrestos
crime_data['LON'] = crime_data['LON'].astype(float)
crime_data['LAT'] = crime_data['LAT'].astype(float)
crime_gs = gpd.GeoSeries([Point(xy) for xy in zip(crime_data['LON'], crime_data['LAT'])], crs="EPSG:4326")

# Reproyectar a un CRS métrico (EPSG:3857)
alcohol_stores_gs = alcohol_stores_gs.to_crs(epsg=3857)
crime_gs = crime_gs.to_crs(epsg=3857)

# Crear cKDTree para una búsqueda rápida del vecino más cercano
def ckdnearest(gdA, gdB):
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    return dist, idx

# Calcular distancias y obtener índices del clúster más cercano
distances, indices = ckdnearest(crime_gs, alcohol_stores_gs)

# Añadir las distancias a la escuela más cercana
crime_data['dist_to_cluster_store'] = distances

# Añadir el clúster más cercano basado en los índices
crime_data['cluster_alcohol_stores'] = alcohol_stores.iloc[indices].reset_index(drop=True)['cluster_k_means']

In [ ]:
crime_data[['LAT', 'LON', 'county subdivision', 'date',  'year', 'month', 'day','crime_gravity', 'dist_to_cluster_store', 'cluster_alcohol_stores', 'Median Household Income', 'Hispanic or Latino']]

,LAT,LON,county subdivision,date,year,month,day,crime_gravity,dist_to_cluster_store,cluster_alcohol_stores,Median Household Income,Hispanic or Latino
0,33.9825,-118.2695,91750,2010-02-20,2010,2,20,3,36.259743,4,45723.0,1193285.0
1,33.9599,-118.3962,91750,2010-09-13,2010,9,13,3,2790.211119,3,45723.0,1193285.0
2,34.0224,-118.2524,91750,2010-08-09,2010,8,9,3,2479.939831,1,45723.0,1193285.0
3,34.1016,-118.3295,91750,2010-01-05,2010,1,5,3,971.955437,1,45723.0,1193285.0
4,34.0387,-118.2488,91750,2010-01-03,2010,1,3,1,733.223012,1,45723.0,1193285.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2993137,34.2790,-118.4116,92785,2023-03-22,2023,3,22,2,5924.253993,2,NaN,NaN
2993138,34.2302,-118.4764,92785,2023-10-02,2023,10,2,3,1363.825645,2,NaN,NaN
2993139,34.0467,-118.2485,91750,2023-07-02,2023,7,2,3,136.677456,1,NaN,NaN
2993140,34.1951,-118.4487,92785,2023-03-05,2023,3,5,3,1550.243783,2,NaN,NaN


**Modelo Final**

Tenemos con varias situaciones, entre ellas, que variables utilizaremos, y que modelo, hacer la prueba con tanta data lo vuelve exageradamente pesado.

In [ ]:
data = crime_data[['LAT',
                   'LON',
                   'Median Household Income',
                   'Hispanic or Latino',
                   'dist_to_cluster_store',
                   'county subdivision',
                   'cluster_alcohol_stores',
                   'crime_gravity']].dropna(subset=['Median Household Income']).sample(100000) # Importante saber que tenemos aprox 3k de rows con Lat y Lon iguales a 0
print(len(data))

# Definir características (features) y etiquetas (labels)
X = data.drop(['crime_gravity'], axis=1)
y = data['crime_gravity']

# Codificar características categóricas
categorical_features = ['county subdivision', 'cluster_alcohol_stores']
numerical_features = X.drop(columns=categorical_features).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

100000


Ajuste de Pesos

In [ ]:
# Método 1: Ajuste de Pesos
pipeline_weighted = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('model', RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42, class_weight='balanced'))])

# Entrenar el modelo con ajuste de pesos
pipeline_weighted.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba con ajuste de pesos
y_pred_weighted = pipeline_weighted.predict(X_test)

# Evaluar el modelo con ajuste de pesos
print("Evaluation report for Random Forest with class_weight='balanced':")
print(classification_report(y_test, y_pred_weighted))
print(f"Accuracy: {accuracy_score(y_test, y_pred_weighted)}")

Evaluation report for Random Forest with class_weight='balanced':
              precision    recall  f1-score   support

           1       0.03      0.02      0.02       191
           2       0.18      0.10      0.13      3163
           3       0.89      0.94      0.92     26646

    accuracy                           0.85     30000
   macro avg       0.37      0.35      0.36     30000
weighted avg       0.81      0.85      0.83     30000

Accuracy: 0.8475


SMOTE (hacer esto con data de criminalidad es realmente correcto?)

In [ ]:
# Método 2: SMOTE
pipeline_smote = ImbPipeline(steps=[('preprocessor', preprocessor),
                                    ('smote', SMOTE(random_state=42)),
                                    ('model', RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42))])

# Entrenar el modelo con SMOTE
pipeline_smote.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba con SMOTE
y_pred_smote = pipeline_smote.predict(X_test)

# Evaluar el modelo con SMOTE
print("Evaluation report for Random Forest with SMOTE:")
print(classification_report(y_test, y_pred_smote))
print(f"Accuracy: {accuracy_score(y_test, y_pred_smote)}")

Evaluation report for Random Forest with SMOTE:
              precision    recall  f1-score   support

           1       0.00      0.01      0.00       191
           2       0.15      0.18      0.17      3163
           3       0.90      0.87      0.88     26646

    accuracy                           0.79     30000
   macro avg       0.35      0.35      0.35     30000
weighted avg       0.81      0.79      0.80     30000

Accuracy: 0.7898666666666667


Best Model

In [ ]:
# Comparar y elegir el mejor modelo
weighted_report = classification_report(y_test, y_pred_weighted, output_dict=True)
smote_report = classification_report(y_test, y_pred_smote, output_dict=True)

# Elegir el modelo basado en la métrica deseada
# Aquí, tomamos el promedio ponderado del f1-score como criterio de selección
weighted_f1 = weighted_report['weighted avg']['f1-score']
smote_f1 = smote_report['weighted avg']['f1-score']

if weighted_f1 > smote_f1:
    best_model = "Weighted Class"
    best_pipeline = pipeline_weighted
    best_f1 = weighted_f1
else:
    best_model = "SMOTE"
    best_pipeline = pipeline_smote
    best_f1 = smote_f1

print(f"The best model is {best_model} with a weighted f1-score of {best_f1:.4f}")

The best model is Weighted Class with a weighted f1-score of 0.8284


In [ ]:
def calcular_probabilidades(lat, lon, county_subdivisions_gdf, alcohol_stores, demographic_data, pipeline):
    # Función para calcular la distancia al punto más cercano
    def ckdnearest(point, gdB):
        nA = np.array([(point.geometry.x[0], point.geometry.y[0])])
        nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
        btree = cKDTree(nB)
        dist, idx = btree.query(nA, k=1)
        return dist[0], idx[0]

    # Reprojectar las coordenadas de entrada
    county_subdivisions_gdf['LON'] = county_subdivisions_gdf['INTPTLON'].astype(float)
    county_subdivisions_gdf['LAT'] = county_subdivisions_gdf['INTPTLAT'].astype(float)
    county_subdivisions_gdf_gs = gpd.GeoSeries(
        [Point(xy) for xy in zip(county_subdivisions_gdf['LON'], county_subdivisions_gdf['LAT'])],
        crs="EPSG:4326"
    ).to_crs(epsg=3857)

    point = gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326").to_crs(epsg=3857)

    # Calcular distancias
    distances, indices = ckdnearest(point, county_subdivisions_gdf_gs)
    distances_alc, indices_alc = ckdnearest(point, alcohol_stores_gs)

    # Obtener datos del punto más cercano
    nearest_data = county_subdivisions_gdf.iloc[indices]
    indices = [0]
    selected_cousubfp = county_subdivisions_gdf.iloc[indices]['COUSUBFP']

    # Filtrar datos demográficos
    filtered_demographic_data = demographic_data[demographic_data['county subdivision'].isin(selected_cousubfp)]
    latest_year_data = filtered_demographic_data.loc[filtered_demographic_data.groupby('county subdivision')['Year'].idxmax()]

    result = latest_year_data[['county subdivision', 'Median Household Income', 'Hispanic or Latino']]
    result['dist_to_cluster_store'] = distances_alc
    result['cluster_alcohol_stores'] = alcohol_stores.iloc[indices]['cluster_k_means'].values
    result['LAT'] = lat
    result['LON'] = lon

    # Preprocesamiento y predicción
    categorical_features = ['county subdivision', 'cluster_alcohol_stores']
    numerical_features = ['LAT', 'LON', 'Median Household Income', 'Hispanic or Latino', 'dist_to_cluster_store']

    # Predicción de probabilidad
    input_data = result[numerical_features + categorical_features]
    probabilities = pipeline.predict_proba(input_data)
    probabilities_df = pd.DataFrame(probabilities, columns=['Probability_1', 'Probability_2', 'Probability_3'])

    return probabilities_df

# Ejemplo de uso de la función
lat = 33.9222131
lon = -118.3337125
probabilidades = calcular_probabilidades(lat, lon, county_subdivisions_gdf, alcohol_stores, demographic_data, best_pipeline)
print(probabilidades)


   Probability_1  Probability_2  Probability_3
0            0.0       0.156065       0.843935


C:\Users\andre\AppData\Local\Temp\ipykernel_18724\3869346668.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['dist_to_cluster_store'] = distances_alc
C:\Users\andre\AppData\Local\Temp\ipykernel_18724\3869346668.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['cluster_alcohol_stores'] = alcohol_stores.iloc[indices]['cluster_k_means'].values
C:\Users\andre\AppData\Local\Temp\ipykernel_18724\3869346668.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from